# Session 2

## Implementing a 2D simulation of Active Brownian Particles (ABP) in C++

In the second session of this tutorial we port the Python code developed in Session 1 to C++. We keep the structure and naming the same as in the Python version and only make changes where necessary to set the sage for the GPU implementation in Session 3.

### Brief overview of the design layout

The image below show key components of a particle based simulation code and how they interact with each other: 

<div align="center">
    
<img src="./layout.png" style="width: 800px;"/>
</div>

Let us briefly focus on the *Evolver* class. Its task is to evolve the system by a single time step, i.e., to propagate the dynamics for $\delta t$. In particular, its task is to:

1. Ensure that the neighbor list is up to date;
2. Perform the integration pre-step;
3. Compute all forces and torques on each particle;
4. Perform the integration post-step;
5. Apply periodic boundary conditions.

In our design, one can have multiple force and torque laws apply in the system. In addition, multiple integrators can act in the same system. 


Here we focus on the item number 3, and in particular the fact that in a given system one could have multiple types of interactions. 

For example, in our case, we apply the self-propulsion force $\mathbf{F}^\text{sp}_i = \alpha \mathbf{n}_i$ on each particle. In addition, each particle experiences soft-core repulsion due to overlaps with its neighbors, i.e. $\mathbf{F}_{i}^{\text{elastic}} = \sum_j k\left(2a - r_{ij}\right)\hat{\mathbf{r}}_{ij}$ for $r_{ij} \le 2a$, with the same meaning of parameters as used in Session 1. In our implementation, each term in the total force is implemented as a separate class. The self-propulsion term is handled by the *SelfPropulsion* class, while the soft-core repulsion is implemented in the *HarmonicForce* class.

The *Evolver* class has member variable called *force_computes* that is a Python list that stores all different types of interactions used in a given simulation. To be a bit more technical, if we want to add self-propulsion and soft-core repulsion to our system, we simply create two objects (instances), one for each of the two ""force type" classes and append them to the *force_computes* list of the *Evolver* class. 

The *evolve* function of the *Evolver* class then **simply** iterates over all elements of the *force_computes* list and calls the appropriate *compute* function for each of element.

It is not a coincidence that we put the word **simply** in bold. Due to Python's expressive and powerful data structure this was all we needed to do. However, a lot of things are happening under the hood here, which we need to understand if we are to switch to C++. Code written in C++ is in general much faster than the same code written in Python. This, however, comes at the expense of lot of things that, colloquially put, just work in Python, need to be explicitly implemented in C++.

In order to understand how to port to C++ the same functionality that allows us to split different types of forces into separate classes, we need to introduce the concepts of *class inheritance* and *smart pointers*.

### Classes

*Class* is the central language feature of C++. It is a user-defined type that represents a concept in the code. An instance of a class is called an *object*. A typical class is, therefore, a piece of C++ code that contains some data (called *member data*) and some operations that are performed on that data (called member functions). In other words, a class is an abstraction of a set of properties and operations that can be performed on those properties. Note that a class can contain members only (i.e., no member functions) but it also can contain no member data and only member functions. At first, the later example sounds paradoxical but it is of central importance for this discussion.  

#### Concrete classes

Let us give several concrete examples of classes. 

One of the central entities in our simulation is a particle. It is, therefore, natural to represent it as a class. This is a very simple class that holds basic information such as position, velocity, type, etc. of a particle. 

```c++
struct Particle
{
    int id;           // particle id
    int type;         // particle type
    real radius;      // particle radius
    real2 r;          // particle position
    real2 v;          // particle velocity 
    real2 f;          // particle force
    real2 n;          // particle director
};
```


<div class="alert alert-block alert-info">
<b>Note:</b> In the previous example, *real* and *real2* are user defined types to be discussed later. We also use *struct* which is essentially a class with all members being *public*, i.e., accessible by non-member functions.  
   
Finally, our actual implementation includes several other member data, which are here omitted for brevity.
</div>

Arguably, *Particle* is not an overly exciting class as it does not do much. A more interesting example is the *System* class. This class hold the list of all particles we would like to simulate and basic operations on them, such as adding a new particle. In a reduced form, this class could look something like this. 

```c++
class System
{
public:
    
    System(Box& box) : _box{box}, _numparticles{0} { _particles.clear(); }
    ~System() { }
    std::vector<Particle>& get(void) { return _particles; }
    void add_particle(const Particle& _particle)
    {
        _particles.push_back(_particle);
        _particles[_numparticles].id = _numparticles;
        _numparticles = _particles.size();
    }
    int size() const { return _numparticles; }
    
private:
    
    Box& _box;                           // Simulation box
    std::vector<Particle> _particles;    // Standard library vector containing all particles
    int _numparticles;                   // Total number of particles in the system
    
    
};
```

<div class="alert alert-block alert-info">
Again, for future compatibility with the GPU our implementation is slightly different. 
</div>

Our *System* class is an example of a *concrete* class. It essentially behaves as any built-in type since it contains the data (e.g., an std vector with particles) and some operations on that data (e.g., adding a new particle.

#### Abstract classes

What about forces though? We gave several examples of forces (e.g., self-propulsion and soft-core repulsion). These are all, however, specific examples of a force. The question is if there is a way to abstract this notion away and instead define an interface to all possible (within reason!) forces, some of which we may want to implement in the future?

The answer to that is provided by the notion of an *abstract class*. An abstract class is an *interface*, i.e., it is impossible to create an object that is an instance of an abstract class. However, it is possible to *inherit* from it and create a *class hierarchy* with *children* classes that share the same interface as the *parent* class and provide as specific implementation. This allows us to create a robust system that can handle the entire class hierarchy and allows adding new functionality without having to make changes to the code that uses his class hierarchy. 

At this point, this all sounds very abstract and overly convoluted. So, the best is to proceed with an example.

Let us create an abstract class called *Force* that will serve as the interface to all forces types (for simplicity, let us restrict to single-particle and pair-wise interactions).

```c++
class Force
{
public:
    
    Force(System& sys) : _sys{sys} { }
    virtual ~Force() { }
    std::string get_name() const { return _name; }
    virtual void compute() = 0;
    
protected:
    
    System& _sys;       // System object 
    std::string _name;  // Name of the interaction type (e.g., harmonic)
};
```

The *Force* class looks like any other class with several important differences. The first one is that two of its member functions have a prefix **virtual**. This means that these functions will be overwritten by children classes and the precise version of the member function which is executed will be determined at the runtime. We will get back to this point shortly. 

In addition, we have this somewhat strange line

```c++
virtual void compute() = 0;
```

the "=0" notation means that the *compute* function is a pure virtual function, i.e., just a name holder. Any attempts to call *Force::compute()* will result in a compilation error. This just reflects the fact that one cannot simply compute *a force* without knowing what that force is. 

Finally, we note that the keyword **protected** means that those data members will be accessible by the children classes. 

#### Class inheritance 

Let us now implement a specific type of interaction, say, self-propulsion. We will do this by defining a class *SelfPropulsionForce* that is a child of the abstract class *Force*.

```c++
class SelfPropulsionForce : public Force
{
public:
    
    SelfPropulsionForce(System& sys, double alpha = 1.0) : Force{sys}, _alpha{alpha} { }
    ~SelfPropulsionForce() override { }
    void compute() override 
    {
        for(auto& p : _sys.get())
        {
           p.f += _alpha*p.n; 
        }
    }
    
private:

    double _alpha;   // strength of the self propulsion force
};

```

In the last example, line
```c++
class SelfPropulsionForce : public Force
```
means that the class *SelfPropulsionForce* is a *child* of the abstract class *Force*, i.e., it shares inherits its interface and some of its properties. The keyword **override** tells to C++ that this class "overrides", i.e., implements the actual computation of, in this case, self-propulsion force.

The power of this lies in how we can use the interface to access the children. 


Let us look into the relevant part of the *Evolver* class, i.e., the past it stores and computes all forces.

```c++
class Evolver
{
public:
    
    //..
    void add_force(const std::string name, std::unique_ptr<Force> f)
    {
        // ...
        _force_list[name] = std::move(f);   // Move is used here to transfer ownership of the unique_ptr f
        // ...
    }
    //...
    void compute_forces()
    {
        for (const auto& force : _force_list)
            force.second->compute();
    }
    // ...
    
private:
    
    std::map<std::string, std::unique_ptr<Force>> _force_list;    // list of all the pointer to the forces
    
};
```

Let us analyze this example. First we note the line
```c++
std::map<std::string, std::unique_ptr<Force>> _force_list;
```
which tells the *Evolver* class to store all force types into a map (i.e., loosely speaking, a C++ version of Python's dictionary) where keys are stings (i.e., name of the force, such as 'self-propulsion') and the value is a **unique pointer** to the *Force* object.

<div class="alert alert-block alert-info">
 <b>Reminder:</b> A pointer is a variable that stores the address of another variable (e.g., an object). A raw pointer is dereferenced (i.e., the content of the memory location it points to is accessed) using the '*' operator. However, when working with *raw* pointers the programmer has to ensure that the memory is released back to the OS once the pointer goes out of scope. Otherwise, one can end up with often hard to track memory leaks.   
</div>

There are two important points in here. A *unique pointer* is a version of a *smart pointer*. A *smart pointer* if a close cousin of the raw pointer with an important difference that it automatically disposes of the object it points to when it goes out of scope. A *unique pointer* is a special type of a smart pointer such that there can be only one copy of it.

The second important point is that we defined a unique pointer to the abstract *Force* class, i.e., this class will serve as the interface to all of its children. Precisely because of this 'place-holder' role we have to use pointers instead of bare types. In other words, a definition such as 
```c++
std::map<std::string, Force> _force_list;
```
would not work, since it is impossible to create object of the abstract type *Force*.

In the main code, we would have a sequence like
```c++
{
    // ...
    Evolver e(/* ... */);
    e.add_force("self-propulsion", std::unique_ptr<SelfPropulsionForce>(new SelfPropulsionForce(2.0)));
    // ...
    e.compute_forces();
    // ...
}
```

Note an important thing here, although the member function 
```c++
void add_force(const std::string name, std::unique_ptr<Force> f)
```
of the *Evolver* class expects 
```c++
std::unique_ptr<Force>
``` 
as it second parameter, in the line
```c++
e.add_force("self-propulsion", std::unique_ptr<SelfPropulsionForce>(new SelfPropulsionForce(2.0)));
``` 
we passed a unique pointer to the *SelfPropulsionForce* class instead. While this looks like a mistake, it is actually intended behavior, since the class *Force* acts as the interface to all implementations of force classes. Since we pass a pointer and not the actual object, the precise implementation of the virtual member functions can be decided at the runtime. Therefore, in this case, Evolver's *compute_force* class will call the *compute* member function defined in the *SelfPropulsionForce* class. 

This mechanics where we can use a base abstract type to refer to a wide range of *derived* (i.e., children) types is called *polymorphism* and is one of the central features of all modern object oriented languages. 

Its power lies in the fact that if in the future we decide to implement, say, a *LennardJonesForce* class we do not need to modify the *Evolve* class at all. Instead, all we need to do would be write 
```c++
{
    // ...
    e.add_force("lennard-jones", std::unique_ptr<JennardJonesForce>(new LennardJonesForce(2.0, 1.0)));
    // ...
    e.compute_forces();
    // ...
}
```
in the main code. 

## Binding C++ and Python